In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import json
import re

class DentalDatasetPreparator:
    def __init__(self, crop_dir='cropped_obb_hemijaws_v2', 
                 patient_csv='cleaned_patient.csv',
                 test_size=0.20, random_state=42):
        self.crop_dir = Path(crop_dir)
        self.patient_csv = patient_csv
        self.test_size = test_size
        self.random_state = random_state
        
    def normalize_name(self, name):
        """Normalize patient name for matching"""
        if pd.isna(name):
            return ""
        # Convert to lowercase, remove extra spaces, keep only alphanumeric
        name = str(name).lower().strip()
        name = re.sub(r'[^a-z0-9\s]', '', name)
        name = re.sub(r'\s+', ' ', name)
        return name
    
    def load_and_match_data(self):
        """Match cropped images with patient metadata"""
        print("🔍 Loading patient metadata...")
        patient_df = pd.read_csv(self.patient_csv)
        
        # Clean column names (remove extra spaces)
        patient_df.columns = patient_df.columns.str.strip()
        
        print(f"📋 Columns found: {list(patient_df.columns)}")
        print(f"📊 Total patients in CSV: {len(patient_df)}")
        
        # Create normalized name column for matching
        patient_df['normalized_name'] = patient_df['Name'].apply(self.normalize_name)
        
        # Show sample data
        print("\n📋 Sample patient data:")
        print(patient_df[['Name', 'DOB', 'Age (Decimal)', 'Gender']].head())
        
        print("\n🔍 Scanning cropped images...")
        crop_files = list(self.crop_dir.glob('*.png'))
        print(f"📂 Found {len(crop_files)} image files")
        
        dataset = []
        unmatched = []
        
        for crop_file in crop_files:
            # Skip CLAHE or raw suffix versions if they exist
            if '_clahe.png' in crop_file.name or '_raw.png' in crop_file.name:
                continue
            
            # Parse filename to extract patient name and side
            if 'mandible_left' in crop_file.name:
                side = 'left'
                patient_name = crop_file.name.replace('_mandible_left.png', '')
            elif 'mandible_right' in crop_file.name:
                side = 'right'
                patient_name = crop_file.name.replace('_mandible_right.png', '')
            else:
                continue
            
            # Normalize the extracted name
            normalized_patient = self.normalize_name(patient_name)
            
            # Try to find match in metadata
            match = patient_df[patient_df['normalized_name'] == normalized_patient]
            
            if len(match) > 0:
                row = match.iloc[0]
                
                # Get age - handle column name variations
                age = row.get('Age (Decimal)', row.get('Age', None))
                if pd.isna(age):
                    print(f"⚠️  Skipping {crop_file.name}: No age data")
                    continue
                
                # Get gender
                gender = row.get('Gender', 'U')  # U for Unknown if missing
                if pd.isna(gender):
                    gender = 'U'
                
                dataset.append({
                    'patient_id': row['Name'],  # Use original name as ID
                    'filename': crop_file.name,
                    'filepath': str(crop_file),
                    'age': float(age),
                    'gender': gender,
                    'side': side,
                    'dob': row.get('DOB', ''),
                    'complete': row.get('Complete', '')
                })
            else:
                unmatched.append(crop_file.name)
        
        df = pd.DataFrame(dataset)
        
        # Report unmatched images
        if unmatched:
            print(f"\n⚠️  {len(unmatched)} images could not be matched to patient data:")
            for fname in unmatched[:10]:  # Show first 10
                print(f"   - {fname}")
            if len(unmatched) > 10:
                print(f"   ... and {len(unmatched) - 10} more")
        
        # Sanity checks
        print("\n" + "="*60)
        print("📊 DATASET SUMMARY")
        print("="*60)
        print(f"✅ Matched images: {len(df)}")
        print(f"❌ Unmatched images: {len(unmatched)}")
        print(f"👥 Unique patients: {df['patient_id'].nunique()}")
        
        images_per_patient = df['patient_id'].value_counts()
        print(f"\n📷 Images per patient distribution:")
        print(images_per_patient.value_counts().sort_index())
        
        # Age distribution
        print("\n🎂 Age distribution:")
        print(f"   Range: {df['age'].min():.1f} - {df['age'].max():.1f} years")
        print(f"   Mean: {df['age'].mean():.2f} ± {df['age'].std():.2f} years")
        print(f"   Median: {df['age'].median():.2f} years")
        
        age_bins = pd.cut(df['age'], bins=[0, 6, 9, 12, 20], 
                         labels=['3-6', '6-9', '9-12', '12+'])
        print("\n   By age group:")
        print(df.groupby(age_bins, observed=True).agg({
            'filename': 'count',
            'age': ['mean', 'std']
        }).round(2))
        
        # Gender distribution
        print("\n⚧ Gender distribution:")
        print(df['gender'].value_counts())
        
        # Side distribution
        print("\n↔️  Side distribution:")
        print(df['side'].value_counts())
        
        return df
    
    def create_age_bins(self, df, n_bins=4):
        """Create stratification bins for age"""
        # Use quantile-based binning for even distribution
        df['age_bin'] = pd.qcut(df['age'], q=n_bins, labels=False, duplicates='drop')
        
        print("\n📦 Age bins created for stratification:")
        for bin_id in sorted(df['age_bin'].unique()):
            bin_data = df[df['age_bin'] == bin_id]
            print(f"  Bin {bin_id}: {bin_data['age'].min():.1f}-{bin_data['age'].max():.1f} years "
                  f"({len(bin_data)} images, {bin_data['patient_id'].nunique()} patients)")
        
        return df
    
    def stratified_patient_split(self, df):
        """
        Split patients (not images) into train/test with stratification
        
        Ensures:
        1. No patient appears in both train and test
        2. Balanced age distribution across splits
        3. Both left and right hemijaws stay together
        """
        print("\n" + "="*60)
        print("🔪 SPLITTING INTO TRAIN/TEST SETS")
        print("="*60)
        
        # Get patient-level statistics
        patient_stats = df.groupby('patient_id').agg({
            'age': 'first',  # Age is same for all images of a patient
            'age_bin': 'first',
            'gender': 'first',
            'filename': 'count'  # Number of images per patient
        }).rename(columns={'filename': 'n_images'})
        
        patient_stats = patient_stats.reset_index()
        
        print(f"👥 Total patients: {len(patient_stats)}")
        print(f"🎯 Target test size: {int(len(patient_stats) * self.test_size)} patients ({self.test_size*100:.0f}%)")
        
        # Stratified split by age bins
        train_patients, test_patients = train_test_split(
            patient_stats['patient_id'],
            test_size=self.test_size,
            random_state=self.random_state,
            stratify=patient_stats['age_bin']
        )
        
        # Create train/test dataframes
        df_train = df[df['patient_id'].isin(train_patients)].copy()
        df_test = df[df['patient_id'].isin(test_patients)].copy()
        
        # Add split column
        df_train['split'] = 'train'
        df_test['split'] = 'test'
        
        # Combine back
        df_full = pd.concat([df_train, df_test], ignore_index=True)
        
        # Validation checks
        print("\n✅ VALIDATION CHECKS:")
        
        # Check 1: No patient overlap
        overlap = set(train_patients) & set(test_patients)
        if len(overlap) == 0:
            print("✅ No patient overlap between train/test")
        else:
            print(f"❌ WARNING: {len(overlap)} patients appear in both splits!")
        
        # Check 2: Age distribution
        print("\n📊 Age distribution comparison:")
        print("\n🏋️ TRAIN SET:")
        train_age_dist = df_train.groupby('age_bin').agg({
            'filename': 'count',
            'age': ['mean', 'std']
        }).round(2)
        print(train_age_dist)
        
        print("\n🧪 TEST SET:")
        test_age_dist = df_test.groupby('age_bin').agg({
            'filename': 'count', 
            'age': ['mean', 'std']
        }).round(2)
        print(test_age_dist)
        
        # Check 3: Sample sizes
        print(f"\n📈 Split summary:")
        print(f"🏋️ Train: {len(df_train)} images, {df_train['patient_id'].nunique()} patients "
              f"({len(df_train)/len(df_full)*100:.1f}%)")
        print(f"🧪 Test:  {len(df_test)} images, {df_test['patient_id'].nunique()} patients "
              f"({len(df_test)/len(df_full)*100:.1f}%)")
        
        return df_full, train_patients, test_patients
    
    def save_splits(self, df_full, train_patients, test_patients, output_dir='dataset_splits'):
        """Save train/test splits to disk"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True, parents=True)
        
        # Save full dataset with split labels
        csv_path = output_path / 'full_dataset_with_splits.csv'
        df_full.to_csv(csv_path, index=False)
        print(f"\n💾 Saved: {csv_path}")
        
        # Save separate train/test CSVs
        df_train = df_full[df_full['split'] == 'train']
        df_test = df_full[df_full['split'] == 'test']
        
        df_train.to_csv(output_path / 'train_set.csv', index=False)
        df_test.to_csv(output_path / 'test_set.csv', index=False)
        print(f"💾 Saved: {output_path / 'train_set.csv'}")
        print(f"💾 Saved: {output_path / 'test_set.csv'}")
        
        # Save patient ID lists
        split_info = {
            'train_patients': sorted(train_patients.tolist()),
            'test_patients': sorted(test_patients.tolist()),
            'random_state': self.random_state,
            'test_size': self.test_size,
            'n_train_patients': len(train_patients),
            'n_test_patients': len(test_patients),
            'n_train_images': len(df_train),
            'n_test_images': len(df_test),
            'age_stats_train': {
                'mean': float(df_train['age'].mean()),
                'std': float(df_train['age'].std()),
                'min': float(df_train['age'].min()),
                'max': float(df_train['age'].max())
            },
            'age_stats_test': {
                'mean': float(df_test['age'].mean()),
                'std': float(df_test['age'].std()),
                'min': float(df_test['age'].min()),
                'max': float(df_test['age'].max())
            }
        }
        
        with open(output_path / 'split_info.json', 'w') as f:
            json.dump(split_info, f, indent=2)
        print(f"💾 Saved: {output_path / 'split_info.json'}")
        
        return df_full
    
    def create_physical_split_folders(self, df_full, output_dir):
        """
        OPTIONAL: Create separate folders with train/test images
        
        WARNING: This duplicates images. Use only if needed for specific training frameworks.
        """
        print("\n" + "="*60)
        print("📁 CREATING PHYSICAL TRAIN/TEST FOLDERS")
        print("="*60)
        
        for split in ['train', 'test']:
            split_dir = Path(output_dir) / split
            split_dir.mkdir(exist_ok=True, parents=True)
            
            split_df = df_full[df_full['split'] == split]
            
            print(f"\n📂 Copying {len(split_df)} images to {split}/")
            
            copied = 0
            for _, row in split_df.iterrows():
                src = Path(row['filepath'])
                dst = split_dir / row['filename']
                
                if src.exists():
                    shutil.copy2(src, dst)
                    copied += 1
            
            print(f"✅ Copied {copied} files")
        
        print("\n✅ Physical folders created!")
        print(f"   📁 {output_dir}/train/")
        print(f"   📁 {output_dir}/test/")
    
    def generate_summary_report(self, df_full, output_dir='dataset_splits'):
        """Generate a detailed summary report"""
        output_path = Path(output_dir)
        
        report = []
        report.append("="*80)
        report.append("DENTAL AGE ESTIMATION DATASET - SPLIT REPORT")
        report.append("="*80)
        report.append(f"Generated: {pd.Timestamp.now()}")
        report.append(f"Random seed: {self.random_state}")
        report.append("")
        
        for split in ['train', 'test']:
            split_df = df_full[df_full['split'] == split]
            
            report.append(f"\n{'='*80}")
            report.append(f"{split.upper()} SET")
            report.append(f"{'='*80}")
            report.append(f"Images: {len(split_df)}")
            report.append(f"Patients: {split_df['patient_id'].nunique()}")
            report.append(f"Age range: {split_df['age'].min():.2f} - {split_df['age'].max():.2f} years")
            report.append(f"Mean age: {split_df['age'].mean():.2f} ± {split_df['age'].std():.2f} years")
            report.append(f"Median age: {split_df['age'].median():.2f} years")
            
            report.append(f"\nAge distribution by bin:")
            age_dist = split_df.groupby('age_bin').agg({
                'filename': 'count',
                'age': ['mean', 'std']
            }).round(2)
            report.append(str(age_dist))
            
            report.append(f"\nGender distribution:")
            report.append(str(split_df['gender'].value_counts()))
            
            report.append(f"\nSide distribution:")
            report.append(str(split_df['side'].value_counts()))
            
            report.append(f"\nImages per patient:")
            img_per_patient = split_df.groupby('patient_id').size()
            report.append(f"  Min: {img_per_patient.min()}")
            report.append(f"  Max: {img_per_patient.max()}")
            report.append(f"  Mean: {img_per_patient.mean():.2f}")
        
        # Save report
        report_text = "\n".join(report)
        with open(output_path / 'split_report.txt', 'w') as f:
            f.write(report_text)
        
        print(f"\n📄 Saved: {output_path / 'split_report.txt'}")
        print("\n" + "="*80)
        print("SUMMARY REPORT")
        print("="*80)
        print(report_text)
    
    def run_complete_pipeline(self, output_dir='dataset_splits', 
                             create_physical_folders=False):
        """
        Execute complete dataset preparation pipeline
        
        Args:
            output_dir: Where to save split files
            create_physical_folders: If True, physically copy images to train/test folders
        """
        print("="*80)
        print("🦷 DENTAL AGE ESTIMATION - DATASET PREPARATION PIPELINE")
        print("="*80)
        
        # Step 1: Load and match data
        df = self.load_and_match_data()
        
        if len(df) == 0:
            print("❌ ERROR: No matched data found!")
            return None
        
        # Step 2: Create age bins for stratification
        df = self.create_age_bins(df, n_bins=4)
        
        # Step 3: Split patients into train/test
        df_full, train_patients, test_patients = self.stratified_patient_split(df)
        
        # Step 4: Save splits
        self.save_splits(df_full, train_patients, test_patients, output_dir)
        
        # Step 5: Create physical folders if requested
        if create_physical_folders:
            self.create_physical_split_folders(df_full, output_dir)
        
        # Step 6: Generate summary report
        self.generate_summary_report(df_full, output_dir)
        
        print("\n" + "="*80)
        print("✅ DATASET PREPARATION COMPLETE!")
        print("="*80)
        print(f"📁 All files saved to: {output_dir}/")
        print("\n📋 Next steps:")
        print("   1. Review split_report.txt for data distribution")
        print("   2. Use train_set.csv for model training with StratifiedGroupKFold")
        print("   3. 🔒 LOCK test_set.csv - do NOT use until final evaluation")
        print("   4. Consider data augmentation for the training set")
        
        return df_full


if __name__ == "__main__":
    # ============================================
    # CONFIGURATION
    # ============================================
    CROP_DIR = 'cropped_obb_hemijaws_v2'  # Your cropped images folder
    PATIENT_CSV = 'cleaned_patient.csv'    # Your patient metadata CSV
    OUTPUT_DIR = 'dataset_splits_2'
    TEST_SIZE = 0.20  # 20% of patients for test set
    RANDOM_STATE = 42  # For reproducibility
    
    # ============================================
    # RUN PIPELINE
    # ============================================
    preparator = DentalDatasetPreparator(
        crop_dir=CROP_DIR,
        patient_csv=PATIENT_CSV,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE
    )
    
    # Create dataset with CSV-only splits (recommended)
    # Set create_physical_folders=True if you need separate train/test folders
    df_final = preparator.run_complete_pipeline(
        output_dir=OUTPUT_DIR,
        create_physical_folders=False  # Set to True to copy files
    )
    
    if df_final is not None:
        print(f"\n✅ Dataset ready!")
        print(f"📊 Total: {len(df_final)} images from {df_final['patient_id'].nunique()} patients")
        print(f"\n🏋️ Training: Use '{OUTPUT_DIR}/train_set.csv'")
        print(f"🧪 Testing: Keep '{OUTPUT_DIR}/test_set.csv' locked until final evaluation")

🦷 DENTAL AGE ESTIMATION - DATASET PREPARATION PIPELINE
🔍 Loading patient metadata...
📋 Columns found: ['Name', 'Date Pic Taken', 'DOB', 'Age (Decimal)', 'Ambiguous', 'Gender', 'Complete']
📊 Total patients in CSV: 483

📋 Sample patient data:
                      Name         DOB  Age (Decimal) Gender
0             Aazkirana. F  01.07.2011           6.74      F
1              ABDULLAH TN  07.11.2010           6.89      M
2  ACHMAD NAUFAL AFANDI у.  03.01.2007          10.05      M
3               ACHMAD NUR  25.05.2007          10.00      M
4   ACHMAD YUSUF MUAFIQ AN  24.04.2008           8.80      M

🔍 Scanning cropped images...
📂 Found 1912 image files

⚠️  956 images could not be matched to patient data:
   - 7_54_ORLIN_AQILA_ATALIA_26082021_mandible_right.png
   - 11_12_IN_AMELINDA_PUTRI_F_10072019_mandible_right.png
   - 12_16_Dinar_Nindytasari_11052018_mandible_right.png
   - 12_57_SILFIANA_DASIFA_16102018_mandible_left.png
   - 11_94_RAHMA_NABILA_281212017_mandible_left.png
   - 

KeyError: 'patient_id'